In [ ]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
# This script captures images from the webcam and saves them to a specified directory.
# It enhances the images by adjusting contrast and sharpness before saving.
# It requires OpenCV, NumPy, and PIL libraries.
# Requirements:
# pip install opencv-python numpy pillow
# Usage:
# python capture_images.py

import cv2
import os
import time
import numpy as np
from PIL import Image, ImageEnhance

def capture_image(name,quantity=20):
    save_dir = f"dataset/{name}/images"
    os.makedirs(save_dir, exist_ok=True)

    count = 1
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open camera.")
        exit()

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv2.CAP_PROP_FPS, 30)

    print("Camera opened successfully.")

    while True and count <= quantity:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(frame)
        frame = ImageEnhance.Contrast(pil_image).enhance(1.5)
        frame = ImageEnhance.Sharpness(frame).enhance(2.0)
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        cv2.imshow('Camera Feed', frame)

        path = f"{save_dir}/{name}_{count}.jpg"
        print(f"Saving image to: {path}")
        cv2.imwrite(path, frame)

        time.sleep(0.3)
        count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

capture_image("face", 200)

In [ ]:
!pip install scikit-learn
!pip install roboflow
!pip install autodistill autodistill-grounded-sam roboflow

In [ ]:
from autodistill.detection import CaptionOntology
from autodistill_grounded_sam import GroundedSAM

def label_faces(name):
    input_path = f"dataset/{name}/images"
    output_path = f"dataset/{name}/output"

    # Use a more descriptive caption prompt for higher accuracy
    ontology = CaptionOntology({"face": "a clear front-facing human face"})

    # Initialize model with the ontology
    model = GroundedSAM(ontology=ontology)

    # Run auto-labeling
    model.label(input_folder=input_path, output_folder=output_path)

label_faces("face")

In [ ]:
# This script moves the labeled files from the output folder to a designated labels folder.
# It also sorts the files numerically based on their names and moves the data.yaml file.
# It requires the os, shutil, and re libraries.

import os
import shutil
import re

def move_files(name):
    output_folder = f"dataset/{name}/output"
    
    # Define source folders for labels
    source_folders = [
        os.path.join(output_folder, "train/labels"),
        os.path.join(output_folder, "valid/labels")
    ]

    # Create target folder for labels
    target_folder = f"dataset/{name}/labels"
    os.makedirs(target_folder, exist_ok=True)

    # Collect and sort all .txt files numerically
    all_txt_files = []
    for folder in source_folders:
        for filename in os.listdir(folder):
            if filename.endswith(".txt"):
                full_path = os.path.join(folder, filename)
                all_txt_files.append(full_path)

    # Sort by the numeric part in filename
    sorted_files = sorted(all_txt_files, key=lambda x: int(re.search(r'\d+', os.path.basename(x)).group()))

    for src in sorted_files:
        filename = os.path.basename(src)
        dst = os.path.join(target_folder, filename)
        shutil.copy2(src, dst)
        print(f"Copied: {filename}")

    # Move data.yaml before deleting output folder
    yaml_path = os.path.join(output_folder, "data.yaml")
    if os.path.exists(yaml_path):
        shutil.move(yaml_path, f"dataset/{name}/data.yaml")
        print("Moved: data.yaml")

    # Delete the output folder
    shutil.rmtree(output_folder)
    print(f"Removed folder: {output_folder}")

# Call the function
move_files("face")



In [ ]:
# This script converts polygon annotations in text files to YOLO format.
# # It reads the coordinates from the text files, calculates the bounding box,
# and writes the YOLO format to the same text files.
# # Requirements:
#   # pip install opencv-python numpy pillow
import os
def convert_polygon_to_yolo_format(label_folder):
    for filename in os.listdir(label_folder):
        if not filename.endswith(".txt"):
            continue

        file_path = os.path.join(label_folder, filename)
        with open(file_path, "r") as file:
            lines = file.read().strip().split()
            if len(lines) < 6:
                continue

            coords = list(map(float, lines[1:]))  # skip class_id
            x_coords = coords[::2]
            y_coords = coords[1::2]

            x_min, x_max = min(x_coords), max(x_coords)
            y_min, y_max = min(y_coords), max(y_coords)

            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            width = x_max - x_min
            height = y_max - y_min

            yolo_format = f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        with open(file_path, "w") as file:
            file.write(yolo_format + "\n")

convert_polygon_to_yolo_format("dataset/face/labels")

In [ ]:
# This script reads YOLO format annotations and draws bounding boxes on images.
## It requires OpenCV, NumPy, and Matplotlib libraries.
# Requirements:
# pip install opencv-python numpy matplotlib

import os
import cv2
import matplotlib.pyplot as plt

def draw_yolo_boxes(image_dir, label_dir, class_name="face"):
    files = [f for f in os.listdir(image_dir) if f.endswith(".jpg") or f.endswith(".png")]

    for file in files:
        image_path = os.path.join(image_dir, file)
        label_path = os.path.join(label_dir, file.rsplit(".", 1)[0] + ".txt")

        if not os.path.exists(label_path):
            print(f"⚠️ No annotation found for {file}")
            continue

        # Read image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width = img.shape[:2]

        # Read annotations
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                class_id, x_center, y_center, w, h = map(float, parts)

                # Convert YOLO to pixel coordinates
                x1 = int((x_center - w / 2) * width)
                y1 = int((y_center - h / 2) * height)
                x2 = int((x_center + w / 2) * width)
                y2 = int((y_center + h / 2) * height)

                # Draw bounding box
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, class_name, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # Show result
        plt.imshow(img)
        plt.title(file)
        plt.axis("off")
        plt.show()
        input("Press Enter to continue...")

# Example usage
draw_yolo_boxes("dataset/face/images", "dataset/face/labels")

In [ ]:
import os
import shutil
import random

def split_dataset(base_dir="dataset/face", train_ratio=0.8):
    image_dir = os.path.join(base_dir, "images")
    label_dir = os.path.join(base_dir, "labels")

    # Create new directories for train/val split
    image_train_dir = os.path.join(image_dir, "train")
    image_val_dir = os.path.join(image_dir, "val")
    label_train_dir = os.path.join(label_dir, "train")
    label_val_dir = os.path.join(label_dir, "val")

    for d in [image_train_dir, image_val_dir, label_train_dir, label_val_dir]:
        os.makedirs(d, exist_ok=True)

    # Get all image filenames
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    image_files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # sort numerically

    # Shuffle and split
    random.shuffle(image_files)
    train_count = int(len(image_files) * train_ratio)
    train_files = image_files[:train_count]
    val_files = image_files[train_count:]

    def move_files(files, image_dst, label_dst):
        for img in files:
            label = img.replace(".jpg", ".txt")
            img_src = os.path.join(image_dir, img)
            lbl_src = os.path.join(label_dir, label)

            img_dst_path = os.path.join(image_dst, img)
            lbl_dst_path = os.path.join(label_dst, label)

            if os.path.exists(img_src) and os.path.exists(lbl_src):
                shutil.move(img_src, img_dst_path)
                shutil.move(lbl_src, lbl_dst_path)
                print(f"Moved: {img} and {label}")

    move_files(train_files, image_train_dir, label_train_dir)
    move_files(val_files, image_val_dir, label_val_dir)

    print("✅ Dataset successfully split into train and val.")

# Run the function
split_dataset()


In [ ]:
import os
import cv2

# ⚙️ Configure your paths
IMAGE_DIRS = [
    "E:/FaceDetection/dataset/face/images/train",
    "E:/FaceDetection/dataset/face/images/val"
]

LABEL_DIRS = [
    "E:/FaceDetection/dataset/face/labels/train",
    "E:/FaceDetection/dataset/face/labels/val"
]

VALID_EXTENSIONS = (".jpg", ".jpeg", ".png")
REMOVE_BROKEN = True  # Set to False if you want to keep broken files for inspection

def validate_dataset(image_dirs, label_dirs):
    for img_dir, lbl_dir in zip(image_dirs, label_dirs):
        print(f"\n📁 Scanning: {img_dir}")
        for filename in os.listdir(img_dir):
            if filename.lower().endswith(VALID_EXTENSIONS):
                image_path = os.path.join(img_dir, filename)
                label_path = os.path.join(lbl_dir, os.path.splitext(filename)[0] + ".txt")

                # Check if image is readable
                img = cv2.imread(image_path)
                if img is None:
                    print(f"❌ Corrupted or unreadable: {image_path}")
                    if REMOVE_BROKEN:
                        os.remove(image_path)
                        print(f"🗑️ Removed: {image_path}")
                        # Optionally delete the label file too
                        if os.path.exists(label_path):
                            os.remove(label_path)
                            print(f"🗑️ Removed label: {label_path}")
                    continue

                # Check if label file exists
                if not os.path.exists(label_path):
                    print(f"⚠️ Missing label file for image: {filename}")

validate_dataset(IMAGE_DIRS, LABEL_DIRS)


In [ ]:
# yolo v8 training
from ultralytics import YOLO
import numpy
model = YOLO("yolov8n.pt")  # Load a pre-trained model
# Train the model on your dataset
model.train(
    data="dataset/face/data.yaml",
    epochs=100,
    imgsz=320,
    batch=16,
    device=0,
    amp=True,
    patience=10,
    save_period=-1,  # Save only best.pt and last.pt
)
  # Adjust parameters as needed

In [ ]:
from ultralytics import YOLO
import cv2

# Load your YOLOv8 model (pre-trained or custom trained)
model = YOLO("runs/detect/train/weights/best.pt")  # replace with "runs/detect/train/weights/best.pt" if you've trained your own

# Open the webcam (0 = default camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Failed to open webcam.")
    exit()

print("✅ Webcam opened. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read frame from webcam.")
        break

    # Perform inference
    results = model(frame, imgsz=640)

    # Draw bounding boxes on the frame
    annotated_frame = results[0].plot()

    # Display the output
    cv2.imshow("YOLOv8 Camera Detection", annotated_frame)

    # Quit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
print("Device Name:", torch.cuda.get_device_name(0)
      )


In [1]:
import cv2
import numpy as np
import onnxruntime as ort
from ultralytics import YOLO

# ==== CONFIG ====
model_path = "runs/detect/train/weights/best.pt"
onnx_model_path = "modelrgb.onnx"
confidence_threshold = 0.55  # ⬅️ Higher = Stricter
padding = 20  # Padding around face for better crop

# ==== FACE SIZE CONSTRAINT (to approximate distance) ====
MIN_FACE_AREA = 8000     # Faces smaller than this are considered far
MAX_FACE_AREA = 40000    # Faces larger than this are suspicious/spoofed

# ==== LOAD MODELS ====
print("[INFO] Loading models...")
yolo_model = YOLO(model_path)
session = ort.InferenceSession(onnx_model_path)

input_name = session.get_inputs()[0].name
_, _, H, W = session.get_inputs()[0].shape
print(f"[INFO] ONNX Input Size: ({H}, {W})")

# ==== PREPROCESS ====
def preprocess_for_liveness(face_img):
    img = cv2.resize(face_img, (W, H))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2, 0, 1))
    return img[None, :]

# ==== LIVENESS ====
def is_real_face(face_img, threshold=confidence_threshold):
    face = preprocess_for_liveness(face_img)
    prediction = session.run(None, {input_name: face})[0]

    # Determine output format
    if prediction.shape[-1] == 2:
        softmax = np.exp(prediction[0]) / np.sum(np.exp(prediction[0]))
        real_score = float(softmax[1])
        print(f"[DEBUG] Softmax: {softmax}")
    elif prediction.shape[-1] == 1:
        raw = float(prediction[0][0])
        real_score = 1 / (1 + np.exp(-raw))
        print(f"[DEBUG] Sigmoid Score: {real_score:.4f}")
    else:
        raise ValueError("Unexpected ONNX output shape")

    is_real = real_score >= threshold
    return is_real, real_score

# ==== VIDEO LOOP ====
cap = cv2.VideoCapture(0)
print("[INFO] Starting camera... Press 'q' to exit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame, verbose=False)[0]

    for box in results.boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)
        face_width = x2 - x1
        face_height = y2 - y1
        face_area = face_width * face_height

        # Filter based on distance using area
        if face_area < MIN_FACE_AREA:
            print(f"[SKIP] Face too far (area: {face_area})")
            continue
        if face_area > MAX_FACE_AREA:
            print(f"[SKIP] Face too large or spoofed (area: {face_area})")
            continue

        # Pad the crop
        h, w = frame.shape[:2]
        x1p = max(0, x1 - padding)
        y1p = max(0, y1 - padding)
        x2p = min(w, x2 + padding)
        y2p = min(h, y2 + padding)
        face = frame[y1p:y2p, x1p:x2p]

        if face.size == 0:
            continue

        # Liveness check
        is_real, score = is_real_face(face)
        label = f"REAL ({score:.2f})" if is_real else f"SPOOF ({score:.2f})"
        color = (0, 255, 0) if is_real else (0, 0, 255)

        # Draw result
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    cv2.imshow("YOLOv8 + Liveness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print("[INFO] Exiting...")
cap.release()
cv2.destroyAllWindows()


[INFO] Loading models...
[INFO] ONNX Input Size: (112, 112)
[INFO] Starting camera... Press 'q' to exit
[DEBUG] Softmax: [    0.33485     0.66515]
[DEBUG] Softmax: [    0.39589     0.60411]
[DEBUG] Softmax: [    0.42257     0.57743]
[DEBUG] Softmax: [    0.40031     0.59969]
[DEBUG] Softmax: [    0.39639     0.60361]
[DEBUG] Softmax: [    0.46404     0.53596]
[DEBUG] Softmax: [    0.33963     0.66037]
[DEBUG] Softmax: [    0.39783     0.60217]
[DEBUG] Softmax: [    0.36243     0.63757]
[DEBUG] Softmax: [    0.41861     0.58139]
[DEBUG] Softmax: [    0.49468     0.50532]
[DEBUG] Softmax: [    0.37725     0.62275]
[DEBUG] Softmax: [    0.41321     0.58679]
[DEBUG] Softmax: [     0.4521      0.5479]
[DEBUG] Softmax: [    0.38368     0.61632]
[DEBUG] Softmax: [    0.34757     0.65243]
[DEBUG] Softmax: [     0.4683      0.5317]
[DEBUG] Softmax: [    0.44006     0.55994]
[DEBUG] Softmax: [     0.5158      0.4842]
[DEBUG] Softmax: [     0.4298      0.5702]
[DEBUG] Softmax: [    0.40775     0.

In [ ]:
# FACE EMBEDDING

In [ ]:
import cv2
import os

# ==== CONFIG ====
person_name = "Anwesha"     # change this to your name or the label
num_images = 30           # number of images to capture
save_dir = f"dataset/{person_name}"
os.makedirs(save_dir, exist_ok=True)

# ==== Initialize webcam ====
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not accessible.")
    exit()

print("[INFO] Capturing images... Press 'q' to quit early.")
count = 0

while count < num_images:
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame.")
        break

    # Optionally, show rectangle on face (you can skip this block)
    cv2.putText(frame, f"Capturing {count+1}/{num_images}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    cv2.imshow("Capture - Press 'q' to quit", frame)

    # Save the frame
    img_path = os.path.join(save_dir, f"{count:03d}.jpg")
    cv2.imwrite(img_path, frame)
    print(f"[INFO] Saved {img_path}")
    count += 1

    # Wait for 100ms or 'q' to quit
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("[INFO] Done capturing images.")

In [ ]:
# This script extracts face embeddings from images using InsightFace.
# It saves the embeddings in a dictionary and stores them in a pickle file.

import os
import cv2
import numpy as np
import pickle
from insightface.app import FaceAnalysis

# === Initialize FaceAnalysis from InsightFace ===
face_app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])  # Use 'cuda' if GPU available
face_app.prepare(ctx_id=0)

# === Path where images are stored ===
image_folder = "dataset"  # Example folder structure: images/person1, images/person2, ...
output_pickle = "face_embeddings.pkl"

# === Dictionary to store embeddings ===
face_embeddings = {}

# === Iterate through each person folder ===
for person_name in os.listdir(image_folder):
    person_folder = os.path.join(image_folder, person_name)
    if not os.path.isdir(person_folder):
        continue

    embeddings = []
    for filename in os.listdir(person_folder):
        filepath = os.path.join(person_folder, filename)
        img = cv2.imread(filepath)
        if img is None:
            continue

        faces = face_app.get(img)
        if not faces:
            continue

        # Take only first detected face (you can modify this for multiple faces)
        emb = faces[0].embedding
        embeddings.append(emb)

    if embeddings:
        # Average embeddings for multiple images of the same person
        face_embeddings[person_name] = np.mean(embeddings, axis=0)

# === Save embeddings to a pickle file ===
with open(output_pickle, 'wb') as f:
    pickle.dump(face_embeddings, f)

print(f"Embeddings saved to {output_pickle}")


In [3]:
import cv2
import numpy as np
import pickle
from ultralytics import YOLO
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity

# === Load YOLOv8 face detection model ===
yolo_model = YOLO("runs/detect/train/weights/best.pt")

# === Load known face embeddings ===
with open("face_embeddings.pkl", "rb") as f:
    known_embeddings = pickle.load(f)  # {name: 512-d array}

print(f"✅ Loaded {len(known_embeddings)} known embeddings: {list(known_embeddings.keys())}")

# === Initialize InsightFace ===
face_app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
face_app.prepare(ctx_id=0)

# === Function to compare face embeddings ===
def identify_person(face_emb, threshold=0.35):
    max_score = 0
    identity = "Unknown"
    for name, known_emb in known_embeddings.items():
        score = cosine_similarity([face_emb], [known_emb])[0][0]
        print(f"🔍 {name} → Score: {score:.4f}")
        if score > max_score and score > threshold:
            max_score = score
            identity = name
    return identity

# === Start webcam feed ===
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)

        # Instead of cropping — run InsightFace on the full frame
        faces = face_app.get(frame)

        name = "Unknown"
        for face in faces:
            # Check if face bbox overlaps with YOLO bbox (basic IOU or center check)
            fx1, fy1, fx2, fy2 = face.bbox.astype(int)
            face_center = ((fx1 + fx2) // 2, (fy1 + fy2) // 2)

            if x1 <= face_center[0] <= x2 and y1 <= face_center[1] <= y2:
                emb = face.embedding
                name = identify_person(emb)
                break  # Use first matching face only

        # Draw box and name
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, name, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("YOLO + InsightFace", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()




✅ Loaded 2 known embeddings: ['Anwesha', 'Vijit']
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\w60

In [1]:
# This script combines YOLOv8 for face detection, InsightFace for face embedding extraction, and ONNX for liveness detection.
# It captures video from the webcam, detects faces, checks liveness, and identifies persons based on pre-computed embeddings.


import cv2
import numpy as np
import pickle
import onnxruntime as ort
from ultralytics import YOLO
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity


def detect_faces():
        

    # ==== CONFIG ====
    model_path = "runs/detect/train/weights/best.pt"
    onnx_model_path = "modelrgb.onnx"
    confidence_threshold = 0.50
    padding = 20
    MIN_FACE_AREA = 8000
    MAX_FACE_AREA = 40000
    embedding_threshold = 0.35

    # ==== LOAD MODELS ====
    print("[INFO] Loading models...")
    yolo_model = YOLO(model_path)
    session = ort.InferenceSession(onnx_model_path)
    input_name = session.get_inputs()[0].name
    _, _, H, W = session.get_inputs()[0].shape

    with open("face_embeddings.pkl", "rb") as f:
        known_embeddings = pickle.load(f)
    print(f"✅ Loaded {len(known_embeddings)} known embeddings: {list(known_embeddings.keys())}")

    face_app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
    face_app.prepare(ctx_id=0)

    # ==== UTILS ====
    def preprocess_for_liveness(face_img):
        img = cv2.resize(face_img, (W, H))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2, 0, 1))
        return img[None, :]

    def is_real_face(face_img, threshold=confidence_threshold):
        face = preprocess_for_liveness(face_img)
        prediction = session.run(None, {input_name: face})[0]

        if prediction.shape[-1] == 2:
            softmax = np.exp(prediction[0]) / np.sum(np.exp(prediction[0]))
            real_score = float(softmax[1])
        elif prediction.shape[-1] == 1:
            raw = float(prediction[0][0])
            real_score = 1 / (1 + np.exp(-raw))
        else:
            raise ValueError("Unexpected ONNX output shape")

        return real_score >= threshold, real_score

    def identify_person(face_emb, threshold=embedding_threshold):
        max_score = 0
        identity = "Unknown"
        for name, known_emb in known_embeddings.items():
            score = cosine_similarity([face_emb], [known_emb])[0][0]
            if score > max_score and score > threshold:
                max_score = score
                identity = name
        return identity

    # ==== VIDEO LOOP ====
    cap = cv2.VideoCapture(0)
    print("[INFO] Starting camera... Press 'q' to exit")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = yolo_model(frame, verbose=False)[0]

        for box in results.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            face_width = x2 - x1
            face_height = y2 - y1
            face_area = face_width * face_height

            if face_area < MIN_FACE_AREA:
                print(f"[SKIP] Face too far (area: {face_area})")
                continue
            if face_area > MAX_FACE_AREA:
                print(f"[SKIP] Face too large or spoofed (area: {face_area})")
                continue

            h, w = frame.shape[:2]
            x1p = max(0, x1 - padding)
            y1p = max(0, y1 - padding)
            x2p = min(w, x2 + padding)
            y2p = min(h, y2 + padding)
            face_crop = frame[y1p:y2p, x1p:x2p]

            if face_crop.size == 0:
                continue

            is_real, score = is_real_face(face_crop)
            liveness_label = f"REAL ({score:.2f})" if is_real else f"SPOOF ({score:.2f})"
            color = (0, 255, 0) if is_real else (0, 0, 255)

            name = "Unknown"
            if is_real:
                faces = face_app.get(frame)
                for face in faces:
                    fx1, fy1, fx2, fy2 = face.bbox.astype(int)
                    center = ((fx1 + fx2) // 2, (fy1 + fy2) // 2)
                    if x1 <= center[0] <= x2 and y1 <= center[1] <= y2:
                        name = identify_person(face.embedding)
                        break

            label = f"{name} | {liveness_label}"
            print(name)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        cv2.imshow("YOLOv8 + Liveness + Face ID", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


detect_faces()


[INFO] Loading models...
✅ Loaded 2 known embeddings: ['Vijit Kumar', 'Gagan']


e:\Users\anaconda3\envs\face\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\win11/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

In [6]:
import pickle
with open("face_embeddings.pkl", "rb") as f:
    known_embeddings = pickle.load(f) 




keys_to_delete = ["TUSHAR", "Aditya"]
for key in keys_to_delete:
    known_embeddings.pop(key, None) 

print(known_embeddings.keys())

with open("face_embeddings.pkl","wb") as f:
    pickle.dump(known_embeddings,f)

dict_keys(['Vijit Kumar', 'Gagan', 'Tushar', 'Vinayak', 'Manjeet Pal', 'Shubhangi', 'Adi'])


In [7]:
import pickle
with open("face_embeddings.pkl", "rb") as f:
    known_embeddings = pickle.load(f) 
print(known_embeddings.keys())


dict_keys(['Vijit Kumar', 'Gagan', 'Tushar', 'Vinayak', 'Manjeet Pal', 'Shubhangi', 'Adi'])


In [ ]:
absl-py==2.3.1
addict==2.4.0
albucore==0.0.24
albumentations==2.0.8
annotated-types==0.7.0
asgiref==3.9.1
asttokens==2.4.1
attrs==25.3.0
autodistill==0.1.29
autodistill-grounded-sam==0.1.2
beautifulsoup4==4.13.4
certifi==2025.7.9
cffi==1.17.1
charset-normalizer==3.4.2
click==8.2.1
cmake==3.29.2
colorama==0.4.6
coloredlogs==15.0.1
comm==0.2.2
contourpy==1.3.2
cvzone==1.6.1
cycler==0.12.1
Cython==3.1.2
debugpy==1.8.1
decorator==5.1.1
defusedxml==0.7.1
Django==4.2.14
djangorestframework==3.16.0
dnspython==2.7.0
drf-yasg==1.21.10
easydict==1.13
exceptiongroup==1.2.1
executing==2.0.1
filelock==3.18.0
filetype==1.2.0
flatbuffers==25.2.10
fonttools==4.58.5
fsspec==2025.5.1
gdown==5.2.0
huggingface-hub==0.33.4
humanfriendly==10.0
idna==3.7
imageio==2.37.0
importlib-metadata==7.1.0
inflection==0.5.1
insightface==0.7.3
ipykernel==6.29.4
ipython==8.24.0
ipywidgets==8.1.7
jax==0.6.2
jaxlib==0.6.2
jedi==0.19.1
Jinja2==3.1.6
joblib==1.5.1
jupyter-client==8.6.1
jupyter-core==5.7.2
jupyterlab-widgets==3.0.15
kiwisolver==1.4.8
lazy_loader==0.4
MarkupSafe==3.0.2
matplotlib==3.10.3
matplotlib-inline==0.1.7
mediapipe==0.10.21
ml_dtypes==0.5.1
mongoengine==0.29.1
mpmath==1.3.0
mysql-connector-python==9.3.0
nest_asyncio==1.6.0
networkx==3.1
numpy==2.2.6
onnx==1.18.0
onnxruntime==1.22.1
opencv-contrib-python==4.11.0.86
opencv-python==4.12.0.88
opencv-python-headless==4.12.0.88
openvino==2024.6.0
openvino-dev==2024.6.0
openvino-telemetry==2025.2.0
opt-einsum==3.3.0
packaging==24.0
pandas==2.3.1
parso==0.8.3
pickleshare==0.7.5
pillow==11.3.0
pillow-avif-plugin==1.5.2
pillow_heif==1.0.0
platformdirs==4.2.1
prettytable==3.16.0
prompt-toolkit==3.0.43
protobuf==4.25.8
psutil==5.9.8
pure-eval==0.2.2
py-cpuinfo==9.0.0
pycocotools==2.0.10
pycparser==2.22
pydantic==2.11.7
pydantic-core==2.33.2
Pygments==2.18.0
pymongo==4.13.2
pyparsing==3.2.3
python-dateutil==2.9.0.post0
python-dotenv==1.1.1
pytz==2025.2
PyYAML==6.0.2
pyzmq==26.0.3
regex==2024.11.6
requests==2.32.4
requests-toolbelt==1.0.0
rf-groundingdino==0.2.0
rf-segment-anything==1.0
roboflow==1.2.1
safetensors==0.5.3
scikit-image==0.25.2
scikit-learn==1.3.2
scipy==1.15.3
seaborn==0.13.2
sentencepiece==0.2.0
simsimd==6.5.0
six==1.16.0
sounddevice==0.5.2
soupsieve==2.7
stack-data==0.6.3
stringzilla==3.12.5
supervision==0.25.1
sympy==1.13.1
threadpoolctl==3.6.0
tifffile==2025.5.10
timm==1.0.17
tokenizers==0.21.2
tomli==2.2.1
torch==2.5.1
torchaudio==2.5.1
torchvision==0.20.1
tornado==6.4
tqdm==4.67.1
traitlets==5.14.3
transformers==4.53.2
typing-inspection==0.4.1
typing-extensions==4.12.1
tzdata==2025.2
ultralytics==8.3.167
ultralytics-thop==2.0.14
uritemplate==4.2.0
urllib3==2.5.0
wcwidth==0.2.13
widgetsnbextension==4.0.14
yapf==0.43.0
zipp==3.18.2
gunicorn